In [1]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.


import csv
import pandas as pd
from collections import Counter
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn import metrics

import pickle
import math
import re
import os
import glob
import numpy as np
np.random.seed(512)

import sys
sys.path.insert(0, '/home/bobotran/Desktop/Homework/cse234/ML-Data-Prep-Zoo/MLFeatureTypeInference/Downstream-Benchmark')
# from Load_Predictions import *
# from Featurize import *

In [2]:
import sortinghat.pylib as pl
pl.rf_Filename

'/home/bobotran/Desktop/Homework/cse234/temp/SortingHatLib/resources/old/RandForest.pkl'

In [3]:
class_map = {
    0: 'numeric',
    1: 'categorical',
    2: 'datetime',
    3: 'sentence',
    4: 'url',
    5: 'embedded-number',
    6: 'list',
    7: 'not-generalizable',
    8: 'context-specific'
}

In [4]:
testdf = pd.read_csv('../../Benchmark-Labeled-Data/data_test.csv')
test_metadata = pd.read_csv('../../Benchmark-Labeled-Data/Metadata/meta_data.csv')

print(len(testdf),len(test_metadata))
test_merged = pd.merge(testdf,test_metadata,on='Record_id')
print(len(test_merged))

# print(test_merged)
y_true = test_merged.y_act.values.tolist()
# print(y_true)
print(len(y_true))

1985 4233
1985
1985


In [5]:
from tqdm.notebook import tqdm

y_pred = []
prv_csv_name,csv_name = '',''
exception_indices = []

for index, row in tqdm(test_merged.iterrows(), total=len(test_merged)):
    col = row['Attribute_name']
    csv_name = '../RawCSVFiles/' + row['name']
    
    try:
        dataDownstream = pd.read_csv(csv_name, usecols=[col], encoding='latin1', low_memory=False)
#         continue
    except ValueError:
        y_pred.append('Object')
        continue
    dataFeaturized = pl.FeaturizeFile(dataDownstream)
    dataFeaturized1 = pl.FeatureExtraction(dataFeaturized)
    # dataFeaturized1 = dataFeaturized1.fillna(0) # Makes categorical accuracy worse
    y_RF = pl.Load_RF(dataFeaturized1)
    
#     y_RF = sortinghat.get_sortinghat_types(dataDownstream)
#     _, y_RF = sortinghat.get_feature_types_as_arff(dataDownstream)
    
    # dataFeaturized = FeaturizeFile(dataDownstream)
    # dataFeaturized1 = ProcessStats(dataFeaturized)
    # dataFeaturized2 = FeatureExtraction(dataFeaturized,dataFeaturized1,0)
    # dataFeaturized2 = dataFeaturized2.fillna(0)
    # y_RF = sortinghat.Load_RF(dataFeaturized2)
    assert len(y_RF) == 1
    y_pred.append(class_map[int(y_RF[0])])
#     y_pred.append(y_RF[0])

  0%|          | 0/1985 [00:00<?, ?it/s]

In [6]:
# y_pred_full = y_pred
# y_pred = [x for x in y_pred_full if x != 'Object']
print(len(y_pred))

1985


In [7]:
from sklearn.metrics import classification_report
import numpy as np

print('Accuracies: ')
print(accuracy_score(y_true, y_pred))
for v in class_map.values():
    one_vs_all_acc = accuracy_score(np.array(y_true) == v, np.array(y_pred) == v)
    print('\t{}: {}'.format(v, one_vs_all_acc))
matrix = confusion_matrix(y_true, y_pred, labels=list(class_map.values()))
print(matrix)
print(classification_report(y_true, y_pred, labels=list(class_map.values()), digits=3))

Accuracies: 
0.8886649874055416
	numeric: 0.9657430730478589
	categorical: 0.947103274559194
	datetime: 0.9929471032745592
	sentence: 0.9869017632241813
	url: 0.9984886649874055
	embedded-number: 0.9843828715365239
	list: 0.9909319899244332
	not-generalizable: 0.9596977329974811
	context-specific: 0.9596977329974811
[[699   4   0   0   0   0   0   1   3]
 [ 10 399   0   5   0   0   1  35   6]
 [  1   2 135   0   0   3   0   0   0]
 [  0   4   0  80   0   0   1   5   2]
 [  0   1   0   1  30   0   0   0   0]
 [  2   5   1   0   0  75   0   1   1]
 [  1   3   0   3   0   4  41   0   5]
 [  6  14   6   3   1   0   0 181   2]
 [ 40  14   1   2   0   0   0   4 124]]
                   precision    recall  f1-score   support

          numeric      0.921     0.989     0.954       707
      categorical      0.895     0.873     0.884       457
         datetime      0.944     0.957     0.951       141
         sentence      0.851     0.870     0.860        92
              url      0.968     0